# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os



# Import API key
from api_keys import geoapify_key
import gmaps
import gmaps.datasets

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,colares,38.7992,-9.4469,11.85,80,6,4.47,PT,1703229083
1,1,port-aux-francais,-49.3500,70.2167,7.18,97,100,8.15,TF,1703229083
2,2,mandalgovi,45.7625,106.2708,-18.66,72,91,2.33,MN,1703229083
3,3,porto novo,6.4965,2.6036,24.30,47,51,1.72,BJ,1703228905
4,4,ca mau,9.1769,105.1500,30.82,51,100,5.34,VN,1703229085


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
 %%capture --no-display

 # Configure the map plot
 map_plot = city_data_df.hvplot.points("Lng", "Lat", 
                                      geo = True, 
                                      size = "Humidity",
                                      scale = 1,
                                      color = "City",
                                      alpha = 0.5,
                                      tiles = "OSM",
                                      frame_width = 700,
                                      frame_height = 500
)



In [4]:
# Display the map

map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27)
                              & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,bredasdorp,-34.5322,20.0403,24.08,54,0,2.56,ZA,1703229092
29,29,hermanus,-34.4187,19.2345,25.45,64,0,0.11,ZA,1703229094
70,70,al ghayzah,16.2079,52.1760,25.64,56,0,0.91,YE,1703229109
127,127,al hazm,16.1641,44.7769,21.36,30,0,1.28,YE,1703229126
490,490,brisas de zicatela,15.8369,-97.0419,22.34,59,0,2.34,MX,1703229242
565,565,costa rica,-23.4167,-54.6500,22.09,77,0,1.64,BR,1703229265
589,589,parham,17.0968,-61.7705,23.35,87,0,4.04,AG,1703229272


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_weather_df.loc[:,['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ''
hotel_df = hotel_df.reset_index()


# Display sample data
hotel_df

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,22,bredasdorp,ZA,-34.5322,20.0403,54,
1,29,hermanus,ZA,-34.4187,19.2345,64,
2,70,al ghayzah,YE,16.2079,52.1760,56,
3,127,al hazm,YE,16.1641,44.7769,30,
4,490,brisas de zicatela,MX,15.8369,-97.0419,59,
5,565,costa rica,BR,-23.4167,-54.6500,77,
6,589,parham,AG,17.0968,-61.7705,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":5
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    
    latitude=row['Lat']
    longitude=row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
hermanus - nearest hotel: Aloe guest house
al ghayzah - nearest hotel: فندق تاج العرب
al hazm - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
costa rica - nearest hotel: No hotel found
parham - nearest hotel: Oceanic View Exclusive Vacation Cottages


,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,22,bredasdorp,ZA,-34.5322,20.0403,54,Victoria Hotel
1,29,hermanus,ZA,-34.4187,19.2345,64,Aloe guest house
2,70,al ghayzah,YE,16.2079,52.1760,56,فندق تاج العرب
3,127,al hazm,YE,16.1641,44.7769,30,No hotel found
4,490,brisas de zicatela,MX,15.8369,-97.0419,59,Casa de Olas
5,565,costa rica,BR,-23.4167,-54.6500,77,No hotel found
6,589,parham,AG,17.0968,-61.7705,87,Oceanic View Exclusive Vacation Cottages


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols=['Humidity','Hotel Name','Country']

)

# Display the map

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)